### CS4423 - Networks
Angela Carnevale <br />
School of Mathematical and Statistical Sciences<br />
NUI Galway

#### 2. Tree and Graph Traversal

# Week 4, lecture 2: Decoding a Pruefer code. Depth and Breadth First Search. 

In [ ]:
import networkx as nx
import numpy as np
opts={"with_labels":True, "node_color":'y'}

Maybe surprisingly, a tree $T$ can be reconstructed from its Prüfer code.  This is based on the following fact
and shows that the map from trees to codes is a bijection!


<b>Fact:</b> The degree of node $x$ is $1$ plus the number of entries $x$ in the Prüfer code of $T$.


Here's our tree from yesterday...

In [ ]:
n=8
nodes=range(n)
edges=[(0,2),(0,5),(0,6),(0,7),(1,7),(3,6),(4,7)]
TT=nx.Graph()
TT.add_nodes_from(nodes)
TT.add_edges_from(edges)
nx.draw(TT,**opts)

... and the Prüfer code we obtained from it

In [ ]:
code = [7,0,6,7,0,0]

In [ ]:
degrees = [1 for k in range(n)]
for k in code:
    degrees[k] += 1
degrees

In [ ]:
[TT.degree[x] for x in TT]

How to restore the tree from its Prüfer code:

* Start with a graph with vertex set $X = \{0, 1, 2, \dots, n-1\}$ (and no edges yet).
* Compute the desired node degrees from the code.
* For each node $y$ in the code find the smallest degree-$1$-node $x$ and
add the edge $x - y$, then decrease the degrees of both $x$ and $y$ by $1$.
* Finally, connect the remaining $2$ nodes of degrees $1$ by an edge.

In [ ]:
T = nx.empty_graph(n)
nx.draw(T, **opts)

In [ ]:
code

In [ ]:
# repeat n-2 times:
for y in code:
    x = degrees.index(1)
    T.add_edge(x, y)
    degrees[x] -= 1;  degrees[y] -= 1
    print(degrees, ": edge", x, "--", y)


Add the final edge:

In [ ]:
e = [x for x in range(n) if degrees[x] == 1]
T.add_edge(*e)
print(e)

In [ ]:
nx.draw(T, **opts)

Turn the entire procedure into a `python` function:

In [ ]:
def tree_pruefer(code):

    # initialize graph and defects
    n = len(code) + 2
    tree = nx.empty_graph(n)
    degrees = [1 for x in tree]
    for y in code:
        degrees[y] += 1
        
    # add edges
    for y in code:
        for x in tree:
            if degrees[x] == 1:
                tree.add_edge(x, y)
                for z in (x, y):
                    degrees[z] -= 1
                break
                
    # final edge
    e = [x for x in tree if degrees[x] == 1]
    tree.add_edge(*e)
    
    return tree

* We can now construct a random tree on $n$ nodes from a random Prüfer code of length $n-2$.

In [ ]:
code = np.random.randint(n, size=n-2)
code

In [ ]:
tree = tree_pruefer(code)
nx.draw(tree, **opts)

Finally, we wrap this up into our own `python` function `random_tree`.

In [ ]:
def random_tree(n):
    code = np.random.randint(n, size=n-2)
    return tree_pruefer(code)

In [ ]:
T = random_tree(20)


In [ ]:
nx.draw(T, **opts)

##  Code Corner

### `python`

* `+=`, `-=`: augmented assignment statements [[doc]](https://docs.python.org/3/reference/simple_stmts.html#augmented-assignment-statements)

### `networkx`

* `connected_components` [[doc]](https://networkx.github.io/documentation/stable/reference/algorithms/component.html)

* `random_tree` [[doc]](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.trees.random_tree.html)

* `copy`: [[doc]](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.copy.html)

* `empty_graph` [[doc]](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.classic.empty_graph.html)

### `numpy`

* `array`: [[doc]](https://numpy.org/doc/stable/reference/generated/numpy.array.html)

* `transpose`: [[doc]](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)

* `random.randint`: [[doc]](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)

## Exercises

1.  A tree $T$ uniquely determines its Prüfer code,
and hence the two nodes that remain after (destructively)
computing the code.   What are those two nodes, in terms of
properties of $T$, or its Prüfer code?

2. What tree has Prüfer code $(0, 1, 2, \dots, n-3)$?

#### 2. Tree and Graph Traversal

# Depth and Breadth First Search. 

Both [DFS](https://en.wikipedia.org/wiki/Depth-first_search)
and [BFS](https://en.wikipedia.org/wiki/Breadth-first_search)
are simple but efficient tree (and graph) traversal algorithms.

## Depth First Search

**DFS**: Given a rooted tree $T$ with root $x$, visit all nodes in the tree.
* $S \gets (x)$
* while $S \neq \emptyset$:
* &nbsp; $y \gets S$.pop() 
* &nbsp; visit($y$) 
* &nbsp; $S$.push($y$.children)

Here $S$ is a [**stack**](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) (LIFO):
$S$.pop() yields the **newest** entry.

In [ ]:
TT = nx.random_tree(10)
nx.draw(TT, **opts)

In [ ]:
T = TT.copy()
x = 0
stack = [x]
while len(stack) > 0:
    y = stack.pop()
    stack.extend(T[y])
    T.remove_node(y)
    print(y, stack)

## Breadth First Search

**BFS**: Given a rooted tree $T$ with root $x$, visit all nodes in the tree.
* $Q \gets (x)$
* while $Q \neq \emptyset$:
* &nbsp; $y \gets Q$.pop() 
* &nbsp; visit($y$) 
* &nbsp; $Q$.push($y$.children)

Here, $Q$ is a [**queue**](https://en.wikipedia.org/wiki/Queue_(abstract_data_type)) (FIFO):
$Q$.pop() yields the **oldest** entry.

In [ ]:
T = TT.copy()
x = 0
queue = [x]
while len(queue) > 0:
    y = queue.pop(0)
    queue.extend(T[y])
    T.remove_node(y)
    print(y, queue)

In [ ]:
nx.draw(TT, **opts)

Many questions on networks concerning distance and connectivity can
be answered by a versatile strategy called **Breadth First Search (BFS)**
which effectively grows a **spanning tree** of the underlying graph.

## Graph Traversal

In [ ]:
n = 10
T = nx.random_tree(n)
nx.draw(T, **opts)

### Alternative Implementations

Both DFS and BFS are more like strategies, rather than specific algorithms.
Different problems might require different implementations.
Sometimes, the stack, or the queue don't have to be made explicit:

* In a recursive implementation, DFS can make use of the (`python`) interpreter's
  **function call stack**.
  
* BFS can take advantage of the fact that some types of lists in a (`python`) `for` loops
  are largely organized as **queues**.


In order to keep track of which nodes have already been visited, we maintain for each node
an attribute `"seen"` that is initially `False`, and becomes `True` when the DFS/BFS visits the node.

In `networkx`, the attributes of a node `x` in a graph `G` are kept in a dictionary `G.nodes[x]`.

In [ ]:
TT = T.copy()
for x in TT:
    TT.nodes[x]['seen'] = False
TT.nodes('seen')

* DFS on a tree:

In [ ]:
def dfs(tree, x):      
    print(x, end=', ') 
    tree.nodes[x]['seen'] = True
    for z in tree[x]:
        if not tree.nodes[z]['seen']:
            dfs(tree, z)    

In [ ]:
dfs(TT, 3)

* BFS on a tree:

In [ ]:
TT = T.copy()
for x in TT:
    TT.nodes[x]['seen'] = False

In [ ]:
Q = [3]
TT.nodes[3]['seen'] = True
for y in Q:
    print(y, end=', ')
    for z in TT[y]:
        if not TT.nodes[z]['seen']:
            Q.append(z)
            TT.nodes[z]['seen'] = True

In [ ]:
nx.draw(T, **opts)

## Shortest Paths

* Recall that a __path__ in a network $G = (X, E)$
is a sequence $p = (x_0, x_1, \dots, x_k)$ of
nodes $x_i \in X$, $i = 0, \dots, k$, such that any
pair of consecutive nodes forms an edge in $G$, i.e.,
$\{x_{i-1}, x_i\} \in E$ for all $i = 1, \dots, k$.

* The __length__ $l(p)$ of the path $p$ is the
number of edges, $l(p) = k$.

* In many practical applications it is of interest to find
for a pair $x, y$ of nodes, one or all the paths form $x$ to $y$
connecting the two nodes with the **fewest number of edges** possible.

* This is a more complex measure on a network than, say, the degree
of a node. And we will need a more complex procedure, that is, an
algorithm, in order to answer such questions systematically.
Let's start with a proper definition.

**Definition.** Let $G = (X, E)$ be a simple graph and let
$x, y \in X$.  Let $P(x, y)$ be the set of all paths from $x$ to $y$.
Then the __distance__ $d(x, y)$ from $x$ to $y$ is
$$d(x, y) = \min \{ l(p) : p \in P(x, y) \},$$
the shortest possible length of a path from $x$ to $y$, and a __shortest path__ from $x$ to $y$ is a path $p \in P(x, y)$ of length $l(p) = d(x, y)$.

The __diameter__ $\mathrm{diam}(G)$ of the network $G$ is the length of the longest shortest path between any two nodes,
$$\mathrm{diam}(G) = \max \{ d(x, y) : x, y \in X \}.$$


##  Code Corner

`python`

* `[].pop` [[doc]](https://docs.python.org/2/tutorial/datastructures.html)

* `[].extend`  [[doc]](https://docs.python.org/2/tutorial/datastructures.html)

### `networkx`

* `remove_node`: [[doc]](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.remove_node.html)


* `edge_subgraph`: [[doc]](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.edge_subgraph.html)